# Monitoramento -  Multiclasses - Não Supervisionado

Um conjunto de dados batch é analizado atraves de duas janelas deslizantes, refência e target, comparando as janelas com a estátitica Jensen-Shannon Divergence e a estaimação feita pelo algoritmo [SPEAR](https://arxiv.org/abs/1908.04240).<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

In [ ]:
# parameters
dataset = "/tmp/data/history.csv" #@param {type:"string"}

# hyperparameters
n_target = 50 #@param {type:"integer",label:"Tamanho da janela alvo"}
n_ref = 5 * n_target #@param {type:"integer",label:"Tamanho da janela de referência"}
K = 3 #@param {type:"integer",label:"Sensibilidade do limite",description:"Quanto mais próximo de 0 mais sensível, quanto mais próximo de 3 mais conservador"}

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(dataset)
scores = []

for column in df.columns:
    if "predict_proba" in column:
        scores.append(df[column].tolist())

y_prob_test = np.array(scores).T
y_prob_test.shape

In [ ]:
y_prob_test

## Calculando distância entre as janelas

Jensen-Shannon Divergence: teste de hipótese qual verifica se o histograma de ambas das distribuições são parecidas/semelhantes a um nível de significancia de 0.05. Nos retorna um valor no intervalo [0, 1] onde 0 seria dizer que são a mesma distribuição e 1 se são completamente diferentes.

In [ ]:
from scipy.stats import entropy
from numpy.linalg import norm

def JSD(P, Q, n_target):
    _P = np.histogram(P, n_target-1)[1] # para ambas ficarem do mesmo tamanho
    _P = _P / norm(_P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

## Algoritimos de estimação interativa de percentis.

Esta métrica de semelhança é comparada com outra, qual chamaremos de limite. O limite é calculado por uma combinação linear da estimação de percentis das janelas de reférencia. Para esta estimação temos as opções:

TDigest: estima e atualiza os percentis a cada nova observação adicionada.

SPEAR: Indicado no paper [aqui](https://arxiv.org/pdf/1908.04240.pdf), este algoritmo iterativo dá mais peso as novas observações e se atualiza. 

In [ ]:
## Algoritmo SPEAR
def UpdatePercentiles(P,X,C):
    n = 100
    
    c_per_bin = C/n
    c_target = (C+1)/n    
    c_this = c_per_bin

    if (X < P[0]):
        P[0] = X
    if (X < P[1]):
        c_this = c_this + 1
        
    for i in range(1, n-1):
        delta_c = c_target - c_this 
        if (delta_c > 0):

            if (P[i+1] - P[i] == 0):
                div = 0.000001 # divisão estava dando 0
                if(X < P[i+1]):
                    rho_next = (1 + c_per_bin) / div 
                else:
                    rho_next = c_per_bin / div
                c_this = rho_next * div
            else:
                if(X < P[i+1]):
                    rho_next = (1 + c_per_bin) / (P[i+1] - P[i])
                else:
                    rho_next = c_per_bin / (P[i+1] - P[i])
                c_this = rho_next * (P[i+1] - P[i])
                
            P[i] = P[i] + delta_c/rho_next
        else:

            if(P[i] - P[i-1] == 0):
                div = 0.000001
                rho_this = c_this / div
            else:
                 rho_this = c_this / (P[i] - P[i-1])
                               
            P[i] = P[i] + delta_c/rho_this
            c_this = c_per_bin - delta_c
            
    if (X > P[n-1]):
        P[n-1] = X
    return P

## Deslisando Janelas e Salvando gráficos

Na metodologia SAMM de janela móvel temos o vetor de steammer, os escores, pós avaliação do método de machine learning escolhido.
Nesta metodologia, temos uma janela maior, conjunto de dados de referência, e uma janela menor, dados a serem validados através da comparação com a janela de referência.
A janela de referência tende a ser de 2 a 5 vezes maior do que a janela alvo(target), isto para que a janela target seja suficientemente pequena para conter anomalias e que estas anomalias não sejam “diluídas” neste conjunto de dados, assim, sendo mais identificáveis.

In [ ]:
import plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
levels = y_prob_test.shape[1]

In [ ]:
for n in range(0, levels):

    stream = y_prob_test[:10000, n]
    # inicailiazando todas as variaveis
    
    alarme_soou = 0
    win_ref = []
    win_target = []
    threshold2 = [] #* n_ref # SPEAR
    dist_jds = []
    P_SPEAR = np.asarray([], dtype=np.float64)
    verdadeiro_percent = np.asarray([], dtype=np.float64)


    X = stream[0]
    stream = np.delete(stream, 0)
    win_ref.append(X)

    alarme_soou = 0
    C = 0
    i = 0

    while stream.size: # tamanho max janelas desse batch

        X = stream[0]
        stream = np.delete(stream, 0)
        C = C + 1

        if (C < n_ref):
            win_ref.append(X)
            # adiciona os primeitos n elementos, quais estão na janela referencia

        elif (C < n_ref + n_target):
            win_target.append(X)
            # atualiza os proximos n+1 até o fim da janela target

        else:     
            # agora que temos win_ref e win_targer devidamente preenchidos posso fazer a medição do JSD
            flag = JSD(win_ref, win_target, n_target) 
            verdadeiro_percent = np.append(flag, verdadeiro_percent )
            verdadeiro_percent = np.sort(verdadeiro_percent )
            
            if (C > n_ref + n_target + 100):
                # só aplico a funação de UpdatePercentile depois qeu já tiver as 100 pesições preenchidas e ordenadas
                UpdatePercentiles(P_SPEAR, flag, C)
                    
                #percentil atualiuzado, posso calcular o threshold
                T_SPEAR = P_SPEAR[75] + K * (P_SPEAR[75] - P_SPEAR[25])
                threshold2.append(T_SPEAR)
                    
                #posso começar a guardar vetor JSD
                dist_jds.append(flag)
                if flag > T_SPEAR:
                    alarme_soou = alarme_soou + 1
#                 else:
#                     alarme_soou = 0
                    
            else:
                # preciso ter 100 elementos nesse vetor
                P_SPEAR = np.append(flag, P_SPEAR)
                P_SPEAR = np.sort(P_SPEAR)
                                       
            
            
            # agora começo a mover a janela
            win_target.append(X)
            aux = win_target.pop(0)

            win_ref.append(aux)
            win_ref.pop(0)



    # Constuindo gráficos  
    threshold2 = np.asarray(threshold2, dtype=np.float64)
    dist_jds = np.asarray(dist_jds, dtype=np.float64)
    axis_x = list(range(0, len(dist_jds)))

    ## Visualização Plot.ly 
    par = make_subplots(rows=3, cols=1)

    trace1 = go.Scatter(
                    x = axis_x,
                    y = dist_jds,
                    mode = "lines",
                    name = "JDS",
                    marker = dict(color = 'rgb(255, 102, 255)'),
                    text = "JDS")
    
    trace2 = go.Scatter(
                    x = axis_x,
                    y = threshold2,
                    mode = "lines",
                    name = "SPEAR",
                    marker = dict(color = 'rgb(102,102,255)'),
                    text = "SPEAR")

    data = [trace1, trace2]

    # definir Labels
    layout = go.Layout(title = 'Grafico de Linhas da categoria '+str(n),
                      xaxis= dict(title= 'Passos da SAMM',ticklen= 5,zeroline= False),
                      yaxis= dict(title= 'Métrica Limite',ticklen= 5,zeroline= False))
    fig = go.Figure(data = data, layout = layout)

    fig.show()